In [288]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog
from cvxopt import matrix, solvers, glpk

In [289]:
input_A = 'input/A.csv'
input_G = 'input/G.csv'
input_b = 'input/b.csv'
input_c = 'input/c.csv'
input_h = 'input/h.csv'

In [290]:
def read(filepath):
    return pd.read_csv(filepath)

In [291]:
A_df = read(input_A)
G_df = read(input_G)
b_df = read(input_b)
c_df = read(input_c)
h_df = read(input_h)

In [292]:
A_df.head()

,id,desc,x1,x2,x3,x4,x5,x6,x7,x8,x9


In [293]:
G_df.head()

,id,desc,x1,x2,x3,x4,x5,x6,x7,x8,x9
0,1,cnt by period,1,1,1,0,0,0,0,0,0
1,2,cnt by period,0,1,1,0,0,0,0,0,0
2,3,cnt by period,0,0,1,1,1,0,0,0,0
3,4,cnt by period,0,0,1,1,1,0,0,0,0
4,5,cnt by period,0,0,0,0,1,1,1,0,0


In [294]:
b_df.head()

,id,desc,val


In [295]:
c_df.head()

,id,desc,val
0,1,"7-9, hour",15
1,2,"7-11, hour",25
2,3,"7-15, full",52
3,4,"11-15, hour",22
4,5,"11-19, full",54


In [296]:
h_df.head()

,id,desc,val
0,1,cnt by period,8
1,2,cnt by period,10
2,3,cnt by period,22
3,4,cnt by period,15
4,5,cnt by period,10


In [297]:
assert len(A_df.columns) - 2 == len(G_df.columns) - 2
cnt_x = len(A_df.columns) - 2
all_x = ['x{}'.format(i) for i in range(1, cnt_x + 1)]
print all_x

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9']


In [298]:
def get_eq_params():
    A = np.array(A_df[all_x])
    b = np.array(b_df['val'])
    c = np.array(c_df['val'])
    
    return A, b, c

In [299]:
A, b, c = get_eq_params()

In [300]:
print A.shape
print A
print b.shape
print b
print c.shape
print c

(0, 9)
[]
(0,)
[]
(9,)
[15 25 52 22 54 24 55 23 16]


In [301]:
def get_neq_params():
    # multiply by -1 because having constraints with >=
    G = -1 * np.array(G_df[all_x])
    h = -1 * np.array(h_df['val'])
    
    return G, h

In [302]:
G, h = get_neq_params()

In [303]:
print G.shape
print G
print h.shape
print h

(21, 9)
[[-1 -1 -1  0  0  0  0  0  0]
 [ 0 -1 -1  0  0  0  0  0  0]
 [ 0  0 -1 -1 -1  0  0  0  0]
 [ 0  0 -1 -1 -1  0  0  0  0]
 [ 0  0  0  0 -1 -1 -1  0  0]
 [ 0  0  0  0 -1 -1 -1  0  0]
 [ 0  0  0  0  0  0 -1 -1  0]
 [ 0  0  0  0  0  0 -1 -1 -1]
 [ 0  0 -1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0 -1  0  0]
 [ 0  0 -3  2 -3  0  0  0  0]
 [ 0  0  0  0 -3  2 -3  0  0]
 [-1  0  0  0  0  0  0  0  0]
 [ 0 -1  0  0  0  0  0  0  0]
 [ 0  0 -1  0  0  0  0  0  0]
 [ 0  0  0 -1  0  0  0  0  0]
 [ 0  0  0  0 -1  0  0  0  0]
 [ 0  0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  0 -1  0  0]
 [ 0  0  0  0  0  0  0 -1  0]
 [ 0  0  0  0  0  0  0  0 -1]]
(21,)
[ -8 -10 -22 -15 -10 -20 -16  -8  -2  -2   0   0   0   0   0   0   0   0
   0   0   0]


In [304]:
if A.shape[0] == 0:
    A = np.array([[0 for _ in range(9)]])
    b = np.array([0])

In [305]:
A = matrix(A, tc='d')
b = matrix(b.T, tc='d')
c = matrix(c.T, tc='d')

G = matrix(G, tc='d')
h = matrix(h.T, tc='d')

In [306]:
status, x = glpk.ilp(c=c, A=A, b=b, I=set(range(cnt_x)), G=G, h=h)
print status
print x
print 'total monthly expenses on salary:', np.array(c.T).dot(np.array(x))[0][0]

optimal
[ 0.00e+00]
[ 7.00e+00]
[ 3.00e+00]
[ 1.30e+01]
[ 6.00e+00]
[ 1.20e+01]
[ 2.00e+00]
[ 1.40e+01]
[ 0.00e+00]

total monthly expenses on salary: 1661.0


In [307]:
# ?glpk.ilp